In [30]:
import os
from libv3.utils import *
import pandas as pd
import json

In [31]:
############ configuration ################
############################################

CODE, BEHAVIOUR, THREAD, VER = get_config()   ### config stored in libv3/exp_config.txt

base_dir = '../trace_data' ### can be replaced with 'csv', 'exe_plot', 'histogram'
log_path = base_dir+f'/{CODE}/{THREAD}_thread/version_{VER}/{BEHAVIOUR}'

print(log_path)

#### file to display
trace_file = 2

print('file number:', trace_file)

CODE: theft_protection
BEHAVIOUR: faulty_data
THREAD: single
VER: 3
../trace_data/theft_protection/single_thread/version_3/faulty_data
file number: 2


In [32]:
######### get paths #######################
paths_log, paths_traces, varlist_path, paths_label = get_paths(log_path)

### remove.Ds_store from all lists
paths_log = [x for x in paths_log if '.DS_Store' not in x]
paths_traces = [x for x in paths_traces if '.DS_Store' not in x]
varlist_path = [x for x in varlist_path if '.DS_Store' not in x]
paths_label = [x for x in paths_label if '.DS_Store' not in x]

paths_log.sort()
paths_traces.sort()
varlist_path.sort()

print(paths_log)
print(paths_traces)
print(varlist_path)
print(paths_label)

['../trace_data/theft_protection/single_thread/version_3/faulty_data/log1-com', '../trace_data/theft_protection/single_thread/version_3/faulty_data/log2-sensor', '../trace_data/theft_protection/single_thread/version_3/faulty_data/log3-bitflip']
['../trace_data/theft_protection/single_thread/version_3/faulty_data/trace1-com', '../trace_data/theft_protection/single_thread/version_3/faulty_data/trace2-sensor', '../trace_data/theft_protection/single_thread/version_3/faulty_data/trace3-bitflip']
['../trace_data/theft_protection/single_thread/version_3/faulty_data/varlist1-com', '../trace_data/theft_protection/single_thread/version_3/faulty_data/varlist2-sensor', '../trace_data/theft_protection/single_thread/version_3/faulty_data/varlist3-bitflip']
['../trace_data/theft_protection/single_thread/version_3/faulty_data/labels/trace1-com_label.json', '../trace_data/theft_protection/single_thread/version_3/faulty_data/labels/trace2-sensor_label.json', '../trace_data/theft_protection/single_thread

In [33]:
############# check varlist is consistent ############
############# only for version 3 ######################

if VER == 3:
    to_number = is_consistent(varlist_path)

    if to_number != False:
        from_number = mapint2var(to_number)


varlist 1 is consistent with varlist 0
{'1-0-main-g_ack': 13, '1-control-update_txmsg-0': 12, '1-0-main-lora': 2, '1-0-main-temperature': 8, '1-0-main-ow': 0, '1-0-main-events': 14, '1-0-main-com_timer': 4, '1-control-update_rxmsg-drop': 15, '1-0-loracom-data': 11, '1-control-readdata-0': 10, '1-0-main-temp': 1, '1-0-main-s': 3, '1-control-updatedata-cls.sensor_data': 9, '1-0-sense-temperature': 7, '1-0-main-i': 6, '1-control-init_timer0-0': 5} {'1-0-main-g_ack': 13, '1-control-update_txmsg-0': 12, '1-0-main-lora': 2, '1-0-main-temperature': 8, '1-0-main-ow': 0, '1-0-main-events': 14, '1-0-main-com_timer': 4, '1-control-update_rxmsg-drop': 15, '1-0-loracom-data': 11, '1-control-readdata-0': 10, '1-0-main-temp': 1, '1-0-main-s': 3, '1-control-updatedata-cls.sensor_data': 9, '1-0-sense-temperature': 7, '1-0-main-i': 6, '1-control-init_timer0-0': 5}


In [34]:
############ Get variable list ######################
sorted_keys = list(from_number.keys())
sorted_keys.sort()
var_list = [from_number[key] for key in sorted_keys]   ### get the variable list
# print(var_list)

## Trace Plots

### Process Traces

In [35]:
########## process the traces ###########
col_data = preprocess_traces(paths_traces, var_list)   ### in the format (trace_name, x_data, y_data, y_labels, trace_path) 

../trace_data/theft_protection/single_thread/version_3/faulty_data/trace1-com trace1-com
../trace_data/theft_protection/single_thread/version_3/faulty_data/trace2-sensor trace2-sensor
../trace_data/theft_protection/single_thread/version_3/faulty_data/trace3-bitflip trace3-bitflip


### Generate plot trace data

In [36]:
############### preprocessing data to plot using plotly ##############
'''
Restructure the data in dictionary with (keys,value) pair :-  (time, timestamps) , (trace_name, trace)
'''
all_df = get_dataframe(col_data)   

In [13]:
all_df[2]

,time,trace3-bitflip
0,1369,0
1,1397,1
2,1418,2
3,1423,3
4,1931,4
...,...,...
2768,533325,15
2769,533330,6
2770,533357,7
2771,534374,8


In [12]:

### plot single trace
for i, df in enumerate(all_df):
    
    if i == trace_file:
        plot_single_trace(df, var_list, with_time=False, is_xticks=True)

In [ ]:
### get timestamp

timestamp = index2timestamp(all_df[0], 227)
print(timestamp)

## Execution Interval Plots

### Preprocess data

In [ ]:
####### preprocessing data ########

#### extract timestamps for each variable and store them in a dictionary along with index values for each variable in event trace

var_timestamps = get_var_timestamps(paths_traces)    #### in format (filename, dict of timestamps and index values)

to_plot = preprocess_variable_plotting(var_timestamps, var_list, from_number, trace_number=trace_file)   ### restructure the data for plotting


### Generate execution interval plots

In [ ]:
### plot the data
plot_execution_interval_single(to_plot, is_xticks=False)

## Plot Labels

In [37]:
### count and prepare labels to plot
'''
labels are of format [index1, index2, timestamp1, timestamp2, class]
'''
class_count = defaultdict(int)
for i, path in enumerate(paths_label):
    label_content = prepare_gt(path)
    ind, ts, cls = label_content
    # print(ind, ts, cls)
    for c in cls:
        class_count[c]+=1
        
    if i == trace_file:
        print(path)
        toplot_gt = label_content

    print(os.path.split(path)[-1], class_count)

    # break
for key, val in class_count.items():
    print(key, val)

trace1-com_label.json defaultdict(<class 'int'>, {1: 22, 2: 3})
trace2-sensor_label.json defaultdict(<class 'int'>, {1: 23, 2: 30})
../trace_data/theft_protection/single_thread/version_3/faulty_data/labels/trace3-bitflip_label.json
trace3-bitflip_label.json defaultdict(<class 'int'>, {1: 24, 2: 36, 3: 68})
1 24
2 36
3 68


defaultdict(<class 'int'>, {1: 22, 2: 3})


In [10]:
### plot
for i, df in enumerate(all_df):
    if i == trace_file:
        plot_single_trace(df, var_list, with_time=False, is_xticks=True, ground_truths=toplot_gt)

## Generate train data

In [ ]:
'''
Add the indices for the interval in 'normal_seq_inter' that show correct behvaiour in the traces. 
The format is as follows:
path_traces = [path1, path2, ...]
normal_seq_inter = [ ( intervals for trace1, ...), 
                    ( intervals for trace2 ...),
                      ...] 
'''

normal_seq_inter = (  ( (0,110), (150,900), (1250,2000), (2050,2300) ),  ### v3, normal trace0
            )

for p, n_inter in zip(paths_traces, normal_seq_inter):
    trace = read_traces(p)
    train_data_path = os.path.join(os.path.dirname(p), 'train_data')

    if not os.path.exists(train_data_path):
        os.makedirs(train_data_path)

    for i, inter in enumerate(n_inter):
        start, end = inter
        # trace[start:end].to_csv(os.path.join(train_data_path, f'interval_{start}_{end}.csv'), index=False)
        json.dump(trace[start:end], open(os.path.join(train_data_path, f'interval_{start}_{end}.json'), 'w'))

In [ ]:
normal_seq_inter[0]